# Calculating the reserve Factors of the panels 

# Please enter your name 

In [67]:
name = "fabian"

# Imports 

In [68]:
import pandas as pd
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.strength import *
from formulas.helpers import *
 

# Constants (Changed based on your data)

In [69]:
personal_data = personal_data_provider(name)
sigma_yield = personal_data[0]
EModulus = personal_data[1]
nu = personal_data[2]
print(f"Your personal data is: sigma_yield = {sigma_yield}, EModulus = {EModulus}, nu = {nu}. Please verify.")

#Panel constants 
length = 750
width = 200


Your personal data is: sigma_yield = 490, EModulus = 65420.46, nu = 0.34. Please verify.


# Open stress values & properties of panels 

In [70]:
#Imoport panel stress file and reformat (OLD VERSION)
# paneldf = pd.read_excel(f'../data/{name}/panel.xlsx', index_col=0)
# paneldf = paneldf.drop(['sigmaZZ', 'sigmaXZ', 'sigmaYZ'],axis=1)
# paneldf = paneldf.reset_index()


########
# New version of importing panel stress file
########

paneldf_v2 = pd.read_csv(f'../data/{name}/panel_v2.csv')
element_component_df = pd.read_csv('../data/element_component_matching.csv')

# Rename the element column to match
#element_component_df.rename(columns={'Element ID': 'element_id'}, inplace=True)
paneldf_v2.rename(columns={'Elements': 'Element ID'}, inplace=True)

# now add the component column to the paneldf_v2 by matching the element_id
paneldf_v2 = add_component_names_to_elements(paneldf_v2, element_component_df)

paneldf_v2.rename(columns={'Loadcase': 'Load Case', 'XX': 'sigmaXX', 'YY': 'sigmaYY', 'XY': 'sigmaXY', 'ZZ': 'sigmaZZ', 'YZ': 'sigmaYZ', 'ZX': 'sigmaXZ'}, inplace=True)
paneldf_v2.drop(columns=['FileID', 'Step', 'Layer', 'sigmaZZ', 'sigmaXZ', 'sigmaYZ'], inplace=True)
paneldf_v2.reset_index(drop=True, inplace=True)
paneldf_v2.set_index('Element ID', inplace=True) # @yannisendr this is possible, since pandas allows non-unique indices, however operations that require an unique index won't work 

# Rename the load case column to match the old one
paneldf_v2['Load Case'] = paneldf_v2['Load Case'].replace({
    1: 'Subcase 1 (LC1)',
    2: 'Subcase 2 (LC2)',
    3: 'Subcase 3 (LC3)',
})

#Import panel properties and reformat
panelPropertiesdf = pd.read_excel(f'../data/{name}/panel_properties.xlsx', index_col=0)
panelPropertiesdf = panelPropertiesdf.reset_index()
panelPropertiesdf = panelPropertiesdf.rename(columns={'elements': 'Element ID'})

paneldf = paneldf_v2

#print(paneldf.head(5))
print(paneldf_v2.head(5))
#print(panelPropertiesdf.head(5))

                  Load Case  sigmaXX  sigmaXY  sigmaYY Component Name
Element ID                                                           
1           Subcase 1 (LC1)   -66.67    26.25    12.75         panel1
2           Subcase 1 (LC1)   -82.97    27.10    20.87         panel1
3           Subcase 1 (LC1)   -67.63    36.97    20.74         panel1
4           Subcase 1 (LC1)   -97.67    34.52     2.42         panel2
5           Subcase 1 (LC1)   -89.50    27.04    25.35         panel2


# Split up the loadcases 

In [71]:
loadCase1df = paneldf[paneldf['Load Case'] == 'Subcase 1 (LC1)']
loadCase2df = paneldf[paneldf['Load Case'] == 'Subcase 2 (LC2)']
loadCase3df = paneldf[paneldf['Load Case'] == 'Subcase 3 (LC3)']
# Add the data to load case 
loadCase1df = pd.merge(loadCase1df, panelPropertiesdf, on='Element ID')
loadCase2df = pd.merge(loadCase2df, panelPropertiesdf, on='Element ID')
loadCase3df = pd.merge(loadCase3df, panelPropertiesdf, on='Element ID')

#Set index back to Element ID 
loadCase1df = loadCase1df.set_index('Element ID')
loadCase2df = loadCase2df.set_index('Element ID')
loadCase3df = loadCase3df.set_index('Element ID')

#Give some overview 
loadCase1df.head(5)

,Load Case,sigmaXX,sigmaXY,sigmaYY,Component Name,mass,thickness
Element ID,,,,,,,
1,Subcase 1 (LC1),-66.67,26.25,12.75,panel1,0.00054,4
2,Subcase 1 (LC1),-82.97,27.10,20.87,panel1,0.00054,4
3,Subcase 1 (LC1),-67.63,36.97,20.74,panel1,0.00054,4
4,Subcase 1 (LC1),-97.67,34.52,2.42,panel2,0.00054,4
5,Subcase 1 (LC1),-89.50,27.04,25.35,panel2,0.00054,4


# Now we need to combine into panels:
-Panel ID (as our ID)
-sigmaXXavg
-sigmaYYavg
-sigmaXYavg
-length
-width
-thickness 


In [72]:
# We can just average the stresses @fabianober Tackle rounding!!!! 
#For load case 1 
panelLC1 = loadCase1df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC1['length'] = [length]*len(panelLC1)
panelLC1['width'] = [width]*len(panelLC1)
panelLC1

#For load case 2 
panelLC2 = loadCase2df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC2['length'] = [length]*len(panelLC2)
panelLC2['width'] = [width]*len(panelLC2)
panelLC2

#For load case 3
panelLC3 = loadCase3df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC3['length'] = [length]*len(panelLC3)
panelLC3['width'] = [width]*len(panelLC3)
panelLC3

# for all LC, round the sigmaXX, sigmaYY, sigmaXY values to 2 decimal places
panelLC1[['sigmaXX', 'sigmaYY', 'sigmaXY']] = panelLC1[['sigmaXX', 'sigmaYY', 'sigmaXY']].round(2)
panelLC2[['sigmaXX', 'sigmaYY', 'sigmaXY']] = panelLC2[['sigmaXX', 'sigmaYY', 'sigmaXY']].round(2)
panelLC3[['sigmaXX', 'sigmaYY', 'sigmaXY']] = panelLC3[['sigmaXX', 'sigmaYY', 'sigmaXY']].round(2)

panelLC1

,sigmaXX,sigmaYY,sigmaXY,thickness,length,width
Component Name,,,,,,
panel1,-72.42,18.12,30.11,4.0,750,200
panel10,-72.42,18.12,30.11,4.0,750,200
panel2,-94.49,18.12,30.01,4.0,750,200
panel3,-94.62,18.12,30.01,4.0,750,200
panel4,-93.96,18.12,30.47,4.0,750,200
panel5,-93.46,18.12,53.25,4.0,750,200
panel6,-93.46,18.12,53.25,4.0,750,200
panel7,-93.96,18.12,30.47,4.0,750,200
panel8,-94.62,18.12,30.01,4.0,750,200


# Now we can apply the functions to the load cases 

In [73]:
panelLC1[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC1.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC2[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC2.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC3[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC3.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC1

,sigmaXX,sigmaYY,sigmaXY,thickness,length,width,k_shear,k_biaxial,Reserve Factor
Component Name,,,,,,,,,
panel1,-72.42,18.12,30.11,4.0,750,200,5.62,5.05,1.57
panel10,-72.42,18.12,30.11,4.0,750,200,5.62,5.05,1.57
panel2,-94.49,18.12,30.01,4.0,750,200,5.62,4.83,1.17
panel3,-94.62,18.12,30.01,4.0,750,200,5.62,4.83,1.17
panel4,-93.96,18.12,30.47,4.0,750,200,5.62,4.84,1.18
panel5,-93.46,18.12,53.25,4.0,750,200,5.62,4.84,1.06
panel6,-93.46,18.12,53.25,4.0,750,200,5.62,4.84,1.06
panel7,-93.96,18.12,30.47,4.0,750,200,5.62,4.84,1.18
panel8,-94.62,18.12,30.01,4.0,750,200,5.62,4.83,1.17


# Combining back into 1 Data Frame to output 

In [74]:
#Drop irrelevant columns
panelLC1 = panelLC1.drop(['length','thickness', 'width'], axis=1)
panelLC2 = panelLC2.drop(['length','thickness', 'width'], axis=1)
panelLC3 = panelLC3.drop(['length','thickness', 'width'], axis=1)
panelLC1.head(3)


,sigmaXX,sigmaYY,sigmaXY,k_shear,k_biaxial,Reserve Factor
Component Name,,,,,,
panel1,-72.42,18.12,30.11,5.62,5.05,1.57
panel10,-72.42,18.12,30.11,5.62,5.05,1.57
panel2,-94.49,18.12,30.01,5.62,4.83,1.17


In [75]:
# Rename columns to be unambigous
panelLC1 = panelLC1.rename(columns={"Reserve Factor": "LC 1 RF", 'sigmaXX':'sigmaXXLC1', 'sigmaYY':'sigmaYYLC1', 'sigmaXY':'sigmaXYLC1', 'k_shear':'k_shearLC1', 'k_biaxial':'k_biaxialLC1'})
panelLC2 = panelLC2.rename(columns={"Reserve Factor": "LC 2 RF", 'sigmaXX':'sigmaXXLC2', 'sigmaYY':'sigmaYYLC2', 'sigmaXY':'sigmaXYLC2', 'k_shear':'k_shearLC2', 'k_biaxial':'k_biaxialLC2'})
panelLC3 = panelLC3.rename(columns={"Reserve Factor": "LC 3 RF", 'sigmaXX':'sigmaXXLC3', 'sigmaYY':'sigmaYYLC3', 'sigmaXY':'sigmaXYLC3', 'k_shear':'k_shearLC3', 'k_biaxial':'k_biaxialLC3'})
panelLC1

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,k_shearLC1,k_biaxialLC1,LC 1 RF
Component Name,,,,,,
panel1,-72.42,18.12,30.11,5.62,5.05,1.57
panel10,-72.42,18.12,30.11,5.62,5.05,1.57
panel2,-94.49,18.12,30.01,5.62,4.83,1.17
panel3,-94.62,18.12,30.01,5.62,4.83,1.17
panel4,-93.96,18.12,30.47,5.62,4.84,1.18
panel5,-93.46,18.12,53.25,5.62,4.84,1.06
panel6,-93.46,18.12,53.25,5.62,4.84,1.06
panel7,-93.96,18.12,30.47,5.62,4.84,1.18
panel8,-94.62,18.12,30.01,5.62,4.83,1.17


In [76]:
outputDf = pd.concat([panelLC1, panelLC2, panelLC3], axis=1)
outputDf

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,k_shearLC1,k_biaxialLC1,LC 1 RF,sigmaXXLC2,sigmaYYLC2,sigmaXYLC2,k_shearLC2,k_biaxialLC2,LC 2 RF,sigmaXXLC3,sigmaYYLC3,sigmaXYLC3,k_shearLC3,k_biaxialLC3,LC 3 RF
Component Name,,,,,,,,,,,,,,,,,,
panel1,-72.42,18.12,30.11,5.62,5.05,1.57,-49.05,-12.0,19.94,5.62,3.04,1.46,-36.21,9.06,180.63,5.62,5.05,0.49
panel10,-72.42,18.12,30.11,5.62,5.05,1.57,-49.05,-12.0,19.94,5.62,3.04,1.46,-36.21,9.06,180.63,5.62,5.05,0.49
panel2,-94.49,18.12,30.01,5.62,4.83,1.17,-62.08,-12.0,19.87,5.62,3.23,1.24,-47.24,9.06,180.05,5.62,4.83,0.47
panel3,-94.62,18.12,30.01,5.62,4.83,1.17,-63.02,-12.0,19.87,5.62,3.24,1.22,-47.31,9.06,180.05,5.62,4.83,0.47
panel4,-93.96,18.12,30.47,5.62,4.84,1.18,-62.89,-12.0,20.18,5.62,3.24,1.22,-46.98,9.06,182.81,5.62,4.84,0.46
panel5,-93.46,18.12,53.25,5.62,4.84,1.06,-62.59,-12.0,35.27,5.62,3.23,1.16,-46.73,9.06,319.51,5.62,4.84,0.17
panel6,-93.46,18.12,53.25,5.62,4.84,1.06,-62.59,-12.0,35.27,5.62,3.23,1.16,-46.73,9.06,319.51,5.62,4.84,0.17
panel7,-93.96,18.12,30.47,5.62,4.84,1.18,-62.89,-12.0,20.18,5.62,3.24,1.22,-46.98,9.06,182.81,5.62,4.84,0.46
panel8,-94.62,18.12,30.01,5.62,4.83,1.17,-63.02,-12.0,19.87,5.62,3.24,1.22,-47.31,9.06,180.05,5.62,4.83,0.47


# Output the file 

In [77]:
outputDf.to_excel(f'../data/{name}/output/processed_e.xlsx')